# Lab 2A: Deploy Team Spokes with Model Access

Deploy **Team Spokes** that connect to the centralized Landing Zone via APIM gateway.


## Step 0: Deploy Models to Landing Zone

Ensure required models are deployed with **correct format per provider**:

| Provider | Models | Format |
|----------|--------|--------|
| OpenAI | gpt-4.1, gpt-4o, model-router | `OpenAI` |
| xAI | grok-3 | `xAI` |
| DeepSeek | DeepSeek-R1 | `DeepSeek` |

In [ ]:
from helpers import get_lz_account, get_existing_deployments, deploy_model, REQUIRED_MODELS

LZ_RG = "foundry-lz-parent"
LZ_ACCOUNT = get_lz_account(LZ_RG)
existing = get_existing_deployments(LZ_RG, LZ_ACCOUNT)

print(f"Landing Zone: {LZ_ACCOUNT}")
print(f"Existing: {existing}\n")

for model in REQUIRED_MODELS:
    if model['name'] in existing:
        print(f"✅ {model['name']}")
    else:
        print(f"🚀 {model['name']} ({model['format']})...", end=" ", flush=True)
        ok, err = deploy_model(LZ_RG, LZ_ACCOUNT, model)
        print("✅" if ok else f"❌ {err}")

## Step 1: Load Configuration

In [ ]:
from helpers import load_env, load_spoke_config, get_principal_id

APIM_URL, APIM_KEY = load_env()
config = load_spoke_config()
PRINCIPAL_ID = get_principal_id()

print(f"APIM: {APIM_URL}")
print(f"Teams: {[s['displayName'] for s in config['spokes']]}")
print(f"Principal: {PRINCIPAL_ID[:8]}...")

## Step 2: Deploy Team Spokes

Each spoke gets its own AI Foundry Account + APIM connection to the Landing Zone.

In [ ]:
from helpers import deploy_spoke, save_deployments

deployed_teams = []

for spoke in config['spokes']:
    print(f"🚀 {spoke['displayName']}...", end=" ", flush=True)
    result = deploy_spoke(spoke, PRINCIPAL_ID, APIM_URL, APIM_KEY)
    if result:
        deployed_teams.append(result)
        print(f"✅ {result['accountName']}")
    else:
        print("❌")

save_deployments(deployed_teams)
print(f"\n✅ Deployed {len(deployed_teams)}/{len(config['spokes'])} teams")

## Step 3: Test Model Access via Agent API

**Key Pattern**: Spoke projects access models via `<connection>/<model>` format using the **Agent/Responses API**.

In [ ]:
!pip install azure-ai-projects==2.0.0b2 azure-identity -q

In [5]:
import re
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition
from helpers import make_agent_name

def extract_response(text):
    """Strip DeepSeek <think> tags and leading whitespace"""
    text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)
    return text.strip()

credential = DefaultAzureCredential()

for team in deployed_teams:
    print(f"\n🏢 {team['displayName']}")
    spoke_cfg = next(s for s in config['spokes'] if s['name'] == team['name'])
    
    for i, endpoint in enumerate(team['projectEndpoints']):
        proj = spoke_cfg['projects'][i]
        print(f"  📁 {proj['name']}: {', '.join(proj['allowedModels'])}")
        
        client = AIProjectClient(credential=credential, endpoint=endpoint)
        openai = client.get_openai_client()
        
        for model in proj['allowedModels']:
            gateway_model = f"{team['connectionName']}/{model}"
            agent_name = make_agent_name(team['name'], proj['name'], model)
            
            try:
                agent = client.agents.create_version(
                    agent_name=agent_name,
                    definition=PromptAgentDefinition(model=gateway_model, instructions="You are a space exploration expert. Answer briefly.")
                )
                resp = openai.responses.create(
                    input="What is the largest planet in our solar system?",
                    extra_body={"agent": {"name": agent.name, "version": agent.version, "type": "agent_reference"}}
                )
                text = extract_response(resp.output_text)
                print(f"     ✅ {model}: {text[:40]}...")
            except Exception as e:
                print(f"     ❌ {model}: {str(e)[:60]}")


🏢 Contoso Ltd
  📁 inventory-ai: gpt-4.1-mini, gpt-4.1, model-router
     ✅ gpt-4.1-mini: The largest planet in our solar system i...
     ✅ gpt-4.1: The largest planet in our solar system i...
     ✅ model-router: The largest planet in our solar system i...

🏢 Fabrikam Inc
  📁 doc-studio: gpt-4o, gpt-4o-mini, grok-3
     ✅ gpt-4o: The largest planet in our solar system i...
     ✅ gpt-4o-mini: The largest planet in our solar system i...
     ✅ grok-3: Jupiter is the largest planet in our sol...

🏢 Woodgrove Bank
  📁 risk-analysis: grok-3, DeepSeek-R1, gpt-4.1-mini
     ✅ grok-3: Jupiter is the largest planet in our sol...
     ✅ DeepSeek-R1: **Jupiter** is the largest planet in our...
     ✅ gpt-4.1-mini: The largest planet in our solar system i...


## Why Agent API? Direct Chat Completions Fail

APIM connections **require** the Agent/Responses API. Direct `chat.completions` won't work:

In [ ]:
team = deployed_teams[0]
spoke_cfg = next(s for s in config['spokes'] if s['name'] == team['name'])
model = spoke_cfg['projects'][0]['allowedModels'][0]
gateway_model = f"{team['connectionName']}/{model}"

client = AIProjectClient(credential=credential, endpoint=team['projectEndpoints'][0])
openai = client.get_openai_client()

print(f"Testing: {gateway_model}\n")

# ❌ Direct chat completions - FAILS
print("❌ chat.completions.create():")
try:
    resp = openai.chat.completions.create(model=gateway_model, messages=[{"role": "user", "content": "Name a planet with rings."}])
    print(f"   {resp.choices[0].message.content[:40]}")
except Exception as e:
    print(f"   Error: {str(e)[:80]}...")

# ✅ Agent + Responses API - WORKS
print("\n✅ Agent + responses.create():")
agent = client.agents.create_version(
    agent_name="demoagent",
    definition=PromptAgentDefinition(model=gateway_model, instructions="You are a space exploration expert. Answer briefly.")
)
resp = openai.responses.create(
    input="Name a planet with rings.",
    extra_body={"agent": {"name": agent.name, "version": agent.version, "type": "agent_reference"}}
)
print(f"   {resp.output_text[:40]}")

## Summary

| What | How |
|------|-----|
| Model format | Use provider-specific: `OpenAI`, `xAI`, `DeepSeek` |
| Gateway model | `<connection-name>/<model-name>` |
| API pattern | Must use Agent + `responses.create()` |
| Agent naming | Alphanumeric only, no hyphens at edges |

**Next**: Lab 2B for direct APIM access without Foundry spoke